<a id="top"></a>
## Eliminating Extra Reports from  Index
2020-11-05

The EDGAR index discussed in [indexReader](#https://github.com/johnlichtenstein/tutorials/blob/master/indexReader.ipynb) has a FormType, CompanyName, CIK, DateFiled, and FileName we can use to filter records or grab data. An example is shown in the table below. 

|    Field    | Example                                     |
|:------------|:--------------------------------------------|
| FormType    | ABS-EE                                      |
| CompanyName | AFS SENSUB CORP.                            |
| CIK         | 1347185                                     |
| DateFiled   | 2019-02-27                                  |
| FileName    | edgar/data/1347185/0001347185-19-000010.txt |

The splitting the FileName by "/", the last two entries are the CIK and the Accession Number. The Accession Number shows up in the text of the document itself, which helps keep things straight. The CIK is the same as the CIK field which also helps keep things straight. As discussed in the indexReader, the FileName is not exactly the name you would need to grab the file, but it provides everything you need to figure out that name. 

Something interesting is that the Accession Number can be reported in multiple CIK. And when this happens it really is the identical document reported in both places, on the same day. What happens is that a company creates a portfolio that gets its oen CIK. The company will report the portfolio one time under its own CIK and then stop, while the portfolio continues reporting. The company CIK will be a lower number for than the portfolio. It's the reports from the CIK that keep reporting that are important for what I am doing. Below we show the situation and some examples, and how to [remove](#filter) those extra entries. 

In [1]:
import pandas as pd
import os
from datetime import datetime as dt
from matplotlib import pyplot as plt

import xiongmao # on pip

### Load saved copy of all ABS-EE filings
DataFrame exported in [indexReader](#https://github.com/johnlichtenstein/tutorials/blob/master/indexReader.ipynb). We count the number of distinct accension numbers by looking at the length of the set, and can see there are fewer accession numbers than files. 

In [2]:
indexR = pd.read_excel("absEEindex.xlsx")  
indexR.shape

(12981, 8)

In [3]:
indexR["docAccNum"] = [x.split("/")[-1].split(".")[0] for x in indexR.FileName]
print ("Number of rows.             : %d" %(len(indexR)))
print ("Number of distinct files    : %d" %(len(set(indexR.FileName))))
print ("Number of distinct docAccNum: %d" %(len(set(indexR.docAccNum))))

Number of rows.             : 12981
Number of distinct files    : 12981
Number of distinct docAccNum: 9716


In [4]:
print (indexR.iloc[0].to_markdown())

|             | 0                                           |
|:------------|:--------------------------------------------|
| FormType    | ABS-EE                                      |
| CompanyName | AFS SENSUB CORP.                            |
| CIK         | 1347185                                     |
| DateFiled   | 2019-02-27                                  |
| FileName    | edgar/data/1347185/0001347185-19-000010.txt |
| yearFiled   | 2019                                        |
| isABSee     | 1                                           |
| monthFiled  | 2                                           |
| docAccNum   | 0001347185-19-000010                        |


<a id="unique"></a>
### Doc Accession Number is *not* Unique
And I had expected it to be, so let's look at how many FileNames or CIK we have per docAccNum.
[top](#top)

In [5]:
fcR = indexR.groupby("docAccNum")["FileName"].nunique().todf(nameL=["numFiles"]) 
ccR = indexR.groupby("docAccNum")["CIK"].nunique().todf(nameL=["numCIK"])
countR = fcR.join(ccR)
countR.head()

,numFiles,numCIK
docAccNum,,
0000929638-17-000295,2,2
0000929638-17-000314,2,2
0000929638-17-000459,1,1
0000929638-17-000514,1,1
0000929638-17-000577,1,1


<a id="pairing"></a>
#### Pairing up
The crosstab below looks at the distribution of the number of filenames by the number of CIK for a docAccNum. We can see there is always only ever one or two, and that when there are two CIK there are two filenames. This suggests that the So, mostly, an accession number is associated with 1 file or CIK, but about $\frac{1}{3}$ of the time there are exactly 2 files and 2 CIK. So about $\frac{1}{2}$ of files are these doubles. 
[top](#top)

In [6]:
countR.xTab(*"numFiles numCIK".split())

numCIK,1,2
numFiles,,
1,6451,0
2,0,3265


<a id="examples"></a>
#### Examples
Let's look at the first and last examples. We can see from the dates that this is not something that just happened in 2016, but has kept happening. For the couple of FileNames looked at, the two CompanyName associated look related. It looks like one is CIK is the company tahat put the portfolio together and another the portfolio.
[top](#top)

In [7]:
test = countR.query("numFiles == 2").head(1).reset_index().iloc[0].docAccNum
indexR.query("docAccNum == @test").transpose()

,2314,2322
FormType,ABS-EE,ABS-EE
CompanyName,BMW AUTO LEASING LLC,BMW Vehicle Lease Trust 2017-1
CIK,1126530,1694920
DateFiled,2017-03-09,2017-03-09
FileName,edgar/data/1126530/0000929638-17-000295.txt,edgar/data/1694920/0000929638-17-000295.txt
yearFiled,2017,2017
isABSee,1,1
monthFiled,3,3
docAccNum,0000929638-17-000295,0000929638-17-000295


In [8]:
test = countR.query("numFiles == 2").tail(1).reset_index().iloc[0].docAccNum
indexR.query("docAccNum == @test").transpose()

,766,1135
FormType,ABS-EE,ABS-EE
CompanyName,Ally Auto Assets LLC,Ally Auto Receivables Trust 2019-4
CIK,1477336,1793667
DateFiled,2020-10-28,2020-10-28
FileName,edgar/data/1477336/0001793667-20-000059.txt,edgar/data/1793667/0001793667-20-000059.txt
yearFiled,2020,2020
isABSee,1,1
monthFiled,10,10
docAccNum,0001793667-20-000059,0001793667-20-000059


<a id="more examples"></a>
#### Collecting Multi Reported docAccNum
For the couple of FileNames looked at, the two CompanyName associated look related. One CIK is the company and another the portfolio. But is it like that for all 3k FileName? First let's filter for the docAccNum that are reported twice.

In [9]:
multiL = list(countR.query("numFiles == 2").index) # list of 2x docAccNum
multiR = indexR.query("docAccNum in @multiL") # indexR of those multis 
print (multiR.shape)
multiR.sort_values("DateFiled docAccNum CIK".split()).head(6)

(6530, 9)


,FormType,CompanyName,CIK,DateFiled,FileName,yearFiled,isABSee,monthFiled,docAccNum
7413,ABS-EE,JP MORGAN CHASE COMMERCIAL MORTGAGE SECURITIES...,1013611,2016-11-29,edgar/data/1013611/0001539497-16-004112.txt,2016,1,11,0001539497-16-004112
7439,ABS-EE,JPMCC Commercial Mortgage Securities Trust 201...,1689111,2016-11-29,edgar/data/1689111/0001539497-16-004112.txt,2016,1,11,0001539497-16-004112
3207,ABS-EE,CITIGROUP COMMERCIAL MORTGAGE SECURITIES INC,1258361,2016-11-29,edgar/data/1258361/0001539497-16-004124.txt,2016,1,11,0001539497-16-004124
3239,ABS-EE,CITIGROUP COMMERCIAL MORTGAGE TRUST 2016-P6,1690255,2016-11-29,edgar/data/1690255/0001539497-16-004124.txt,2016,1,11,0001539497-16-004124
2482,ABS-EE,Banc of America Merrill Lynch Commercial Mortg...,1005007,2016-11-29,edgar/data/1005007/0001539497-16-004148.txt,2016,1,11,0001539497-16-004148
7962,ABS-EE,Morgan Stanley Bank of America Merrill Lynch T...,1690577,2016-11-29,edgar/data/1690577/0001539497-16-004148.txt,2016,1,11,0001539497-16-004148


<a id="relation"></a>
#### Lower CIK is Company
In the examples above, we see the lower CIK is "BIG BANK" and the higher CIK "BIG BANK xyz". Let's look at the relationship between the larger and smaller CIK. We will show that the larger CIK in a pair will always be 
associated with only one smaller one (the relationship between them is a function). 

[top](#top)

In [10]:
mmR = multiR.groupby("docAccNum").CIK.min().todf(nameL=["minCIK"]) \
    .join(multiR.groupby("docAccNum").CIK.max().todf(nameL=["maxCIK"]))
print (mmR.shape)
mmR.head()

(3265, 2)


,minCIK,maxCIK
docAccNum,,
0000929638-17-000295,1126530,1694920
0000929638-17-000314,1126530,1694920
0000929638-17-000769,1126530,1716665
0000929638-18-000030,1136586,1725617
0000929638-18-000815,1126530,1753585


<a id="function"></a>
##### Is relationship between maxCIK and minCIK a function?
Yes! If a CIK is associated with a smaller CIK, there is only one. 
[top](#top)

In [11]:
mmR.groupby("maxCIK").minCIK.nunique().todf(nameL=["numMinCIK"]).numMinCIK \
    .value_counts().reset_index().todf(nameL="numMinCIK frequency".split())

,numMinCIK,frequency
0,1,395


##### Between minCIK and maxCIK?
No. If a CIK is associated with a larger CIK, there could be others. As many as 32!

In [12]:
mmR.groupby("minCIK").maxCIK.nunique().todf(nameL=["numMaxCIK"]).numMaxCIK \
    .value_counts().sort_index().reset_index().todf(nameL="numMaxCIK frequency".split())

,numMaxCIK,frequency
0,1,2
1,2,3
2,3,3
3,4,2
4,5,1
5,7,4
6,8,1
7,9,2
8,11,2
9,12,3


#### Do these maxCIK ever appear alone
Do the maxCIK from the multis ever appear alone in the indexR? Yes. For about half of the Accession Numbers reported by CIK that have multiple CIK, there is only one CIK. Conditional probability can be ard to translate well to English so let's discuss the steps to get this count. 
1. Obtain the set of all CIK that are the max of a pair with another CIK
2. Obtain the set of Accession Number for all the entries with those CIK
3. Pull all records for those Accession Numbers
4. Get a frequency discribution of the number of CIK

[top](#top)

In [13]:
max2minR = mmR.reset_index()["maxCIK minCIK".split()].drop_duplicates(subset="maxCIK")
print (max2minR.shape)
max2minR.head()

(395, 2)


,maxCIK,minCIK
0,1694920,1126530
2,1716665,1126530
3,1725617,1136586
4,1753585,1126530
5,1762228,1131131


In [14]:
maxCIKPairedS = set(max2minR.maxCIK) # the set of the maxCIK from the paired
fromMaxAccS = set(indexR.query("CIK in @maxCIKPairedS").docAccNum) # doc acc num for those cik
print (list(map(len, (maxCIKPairedS, fromMaxAccS))))
fromMaxR = indexR.query("docAccNum in @fromMaxAccS") \
    .set_index("docAccNum").join(countR) # corresponding df and counts
fromMaxR.shape

[395, 9616]


(12881, 10)

In [15]:
fromMaxR.numCIK.value_counts()

2    6530
1    6351
Name: numCIK, dtype: int64

#### The smaller CIK does not report as often as the larger
For CIK that are associated with a smaller CIK, about half the time they report alone and about half with the lower CIK. Let's look at an example. First a list of a few entries where the Accession Number has only one CIK. The names look like one of the portfolio names, as we would expect. 

In [16]:
fromMaxR.query("numCIK == 1").sort_values("DateFiled CIK".split()).head(3)

,FormType,CompanyName,CIK,DateFiled,FileName,yearFiled,isABSee,monthFiled,numFiles,numCIK
docAccNum,,,,,,,,,,
0001056404-17-000033,ABS-EE,CFCRE 2016-C7 Mortgage Trust,1690110,2017-01-25,edgar/data/1690110/0001056404-17-000033.txt,2017,1,1,1,1
0001056404-17-000042,ABS-EE,JPMCC Commercial Mortgage Securities Trust 201...,1689111,2017-01-26,edgar/data/1689111/0001056404-17-000042.txt,2017,1,1,1,1
0000950131-17-000002,ABS-EE,CITIGROUP COMMERCIAL MORTGAGE TRUST 2016-P6,1690255,2017-01-27,edgar/data/1690255/0000950131-17-000002.txt,2017,1,1,1,1


#### A Few Examples
The lower CIK typically reports just once. Below we show three examples. Just the top few lines will print here in the notebook, but the full listing for these three will be saved in a [spreadsheet](https://drive.google.com/file/d/1qRuqEnE7E3oJjMykNKqzGKlxgcwm9juw/view?usp=sharing).

In [17]:
wb = pd.ExcelWriter("cikGroupExamples.xlsx")

In [18]:
firstCIK = fromMaxR.query("numCIK == 1").sort_values("DateFiled CIK".split()).iloc[0].CIK # first row above CIK
accNumS = set(fromMaxR.query("CIK == @firstCIK").index) # associated accNum
print (len(accNumS))
exampleR = indexR.query("docAccNum in @accNumS") \
    .sort_values("DateFiled docAccNum CIK".split()).set_index("docAccNum")
print (exampleR.shape)
exampleR.to_excel(wb, sheet_name="%d" %(firstCIK))
exampleR.head(3)

47
(48, 8)


,FormType,CompanyName,CIK,DateFiled,FileName,yearFiled,isABSee,monthFiled
docAccNum,,,,,,,,
0001539497-16-004174,ABS-EE,"CCRE Commercial Mortgage Securities, L.P.",1515166,2016-11-30,edgar/data/1515166/0001539497-16-004174.txt,2016,1,11
0001539497-16-004174,ABS-EE,CFCRE 2016-C7 Mortgage Trust,1690110,2016-11-30,edgar/data/1690110/0001539497-16-004174.txt,2016,1,11
0001056404-17-000033,ABS-EE,CFCRE 2016-C7 Mortgage Trust,1690110,2017-01-25,edgar/data/1690110/0001056404-17-000033.txt,2017,1,1


In [19]:
firstCIK = fromMaxR.query("numCIK == 1").sort_values("DateFiled CIK".split()).iloc[1].CIK # second row above CIK
accNumS = set(fromMaxR.query("CIK == @firstCIK").index) # associated accNum
print (len(accNumS))
exampleR = indexR.query("docAccNum in @accNumS").sort_values("DateFiled docAccNum CIK".split())
exampleR.to_excel(wb, sheet_name="%d" %(firstCIK))
print (exampleR.shape)
exampleR.head()

48
(50, 9)


,FormType,CompanyName,CIK,DateFiled,FileName,yearFiled,isABSee,monthFiled,docAccNum
7413,ABS-EE,JP MORGAN CHASE COMMERCIAL MORTGAGE SECURITIES...,1013611,2016-11-29,edgar/data/1013611/0001539497-16-004112.txt,2016,1,11,0001539497-16-004112
7439,ABS-EE,JPMCC Commercial Mortgage Securities Trust 201...,1689111,2016-11-29,edgar/data/1689111/0001539497-16-004112.txt,2016,1,11,0001539497-16-004112
7414,ABS-EE/A,JP MORGAN CHASE COMMERCIAL MORTGAGE SECURITIES...,1013611,2016-12-21,edgar/data/1013611/0001539497-16-004367.txt,2016,1,12,0001539497-16-004367
7440,ABS-EE/A,JPMCC Commercial Mortgage Securities Trust 201...,1689111,2016-12-21,edgar/data/1689111/0001539497-16-004367.txt,2016,1,12,0001539497-16-004367
7441,ABS-EE,JPMCC Commercial Mortgage Securities Trust 201...,1689111,2017-01-26,edgar/data/1689111/0001056404-17-000042.txt,2017,1,1,0001056404-17-000042


In [20]:
firstCIK = fromMaxR.query("numCIK == 1").sort_values("DateFiled CIK".split()).iloc[-1].CIK # last pair, unpair
accNumS = set(fromMaxR.query("CIK == @firstCIK").index) # associated accNum
print (len(accNumS))
exampleR = indexR.query("docAccNum in @accNumS").sort_values("DateFiled docAccNum CIK".split())
exampleR.to_excel(wb, sheet_name="%d" %(firstCIK))
print (exampleR.shape)
exampleR.head()

3
(5, 9)


,FormType,CompanyName,CIK,DateFiled,FileName,yearFiled,isABSee,monthFiled,docAccNum
11473,ABS-EE,WELLS FARGO COMMERCIAL MORTGAGE SECURITIES INC,850779,2020-09-16,edgar/data/850779/0001539497-20-001187.txt,2020,1,9,0001539497-20-001187
2024,ABS-EE,BANK 2020-BNK28,1822166,2020-09-16,edgar/data/1822166/0001539497-20-001187.txt,2020,1,9,0001539497-20-001187
11474,ABS-EE/A,WELLS FARGO COMMERCIAL MORTGAGE SECURITIES INC,850779,2020-09-30,edgar/data/850779/0001539497-20-001221.txt,2020,1,9,0001539497-20-001221
2025,ABS-EE/A,BANK 2020-BNK28,1822166,2020-09-30,edgar/data/1822166/0001539497-20-001221.txt,2020,1,9,0001539497-20-001221
2026,ABS-EE,BANK 2020-BNK28,1822166,2020-11-02,edgar/data/1822166/0001056404-20-011782.txt,2020,1,11,0001056404-20-011782


In [21]:
wb.close()

In [22]:
pwd

'/Users/jcl/dev/edgar'

<a id="filter"></a>
### Filtering
Now that the situation is clear, there are two things we need to simplify the index
1. The function relating maxCIK to their minCIK, which we can get easily from the table developed in the section where we reviewed the [relation](#relation). 
2. The filtered index with the minCIK removed. 

First the function. This is simple.

[top](top)

In [23]:
functionR = mmR.groupby("maxCIK").head(1)["maxCIK minCIK".split()] # tail(1) would have worked too
functionR.shape

(395, 2)

#### Partition to filter
To filter let's 
1. Ideltify all records belonging to maxCIK, and set those aside in maxIndR
2. Remove from the indexR all docAccNum in maxIndR and put in soloIndR
3. Concatenate maxIndR and soloIndR for the filtIndR
We save a [workbook](https://drive.google.com/file/d/1ZN0RHDjSZUeiW7lRsWi_CIkLYjmr0juh/view?usp=sharing) with the unfiltered, max2min function, maxInd, soloInd, and filtInd as examples. 
[top](#top)

In [24]:
maxIndR = indexR.query("CIK in @functionR.maxCIK").set_index("docAccNum")
maxIndR.shape

(9616, 8)

In [25]:
soloIndR = indexR.query("docAccNum not in @maxIndR.index").set_index("docAccNum")
soloIndR.shape

(100, 8)

In [26]:
filtIndR = pd.concat((maxIndR, soloIndR)).sort_index()
filtIndR.shape

(9716, 8)

In [27]:
wb = pd.ExcelWriter("absEEindex.filtered.xlsx")
indexR.to_excel(wb, sheet_name="unfiltered", index=False)
functionR.to_excel(wb, sheet_name="max2min")
maxIndR.to_excel(wb, sheet_name="maxInd")
soloIndR.to_excel(wb, sheet_name="soloInd")
filtIndR.to_excel(wb, sheet_name="filtInd")
wb.close()

#### End
[top](#top)